## This example demonstrates XCSF hyperparameter tuning

In [1]:
import json

import numpy as np
import optuna
from optuna.distributions import FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.samplers import TPESampler
from optuna.visualization import plot_param_importances
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xcsf

RANDOM_STATE: int = 1

np.random.seed(RANDOM_STATE)

### Load training and test data
Loads the kin8nm dataset from: https://www.openml.org/d/189

In [2]:
data = fetch_openml(data_id=189, as_frame=True, parser="auto")

data.frame.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
0,-0.015119,0.360741,0.469398,1.309675,0.988024,-0.025493,0.664071,0.062763,0.536524
1,0.360478,-0.301395,0.629183,-1.440146,-0.741637,-1.196749,-1.038444,-0.717461,0.308014
2,1.563238,-1.294753,0.078987,1.432937,1.149136,-1.292140,1.562988,-0.937731,0.518900
3,0.199485,0.901157,-1.356304,-0.080525,-0.976628,0.829894,-0.855649,0.930630,0.494151
4,0.659737,0.120552,-0.008756,0.648839,0.626832,-0.646539,1.318074,-0.899172,0.470218


In [3]:
data.frame.describe()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.009351,-0.007233,0.022792,-0.012945,0.007886,-0.009698,0.008402,0.001227,0.714283
std,0.904867,0.901496,0.900765,0.899186,0.906874,0.913243,0.909724,0.906098,0.263607
min,-1.570611,-1.570681,-1.570669,-1.570295,-1.570664,-1.570254,-1.570643,-1.568904,0.040165
25%,-0.770328,-0.778319,-0.749718,-0.780602,-0.770492,-0.802084,-0.776870,-0.777432,0.523584
50%,-0.002010,-0.015152,0.041346,-0.013617,0.009702,-0.003575,0.001338,0.002007,0.708822
75%,0.791179,0.779145,0.792391,0.760165,0.791203,0.781849,0.798014,0.780927,0.901354
max,1.570753,1.570657,1.570736,1.570169,1.570599,1.570597,1.570522,1.570237,1.458521


In [4]:
# numpy
X_orig = np.asarray(data.data, dtype=np.float64)
y_orig = np.asarray(data.target, dtype=np.float64)

# normalise inputs (zero mean and unit variance)
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X_orig)

# reshape outputs into 2D arrays
if y_orig.ndim == 1:
    y_orig = y_orig.reshape(-1, 1)

# normalise outputs (zero mean and unit variance)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(y_orig)

# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

# 10% of training for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

print(f"X_train shape = {np.shape(X_train)}")
print(f"y_train shape = {np.shape(y_train)}")
print(f"X_val shape = {np.shape(X_val)}")
print(f"y_val shape = {np.shape(y_val)}")
print(f"X_test shape = {np.shape(X_test)}")
print(f"y_test shape = {np.shape(y_test)}")

X_DIM: int = np.shape(X_train)[1]
Y_DIM: int = np.shape(y_train)[1]

X_train shape = (6634, 8)
y_train shape = (6634, 1)
X_val shape = (738, 8)
y_val shape = (738, 1)
X_test shape = (820, 8)
y_test shape = (820, 1)


### Initialise XCSF

In [5]:
# Due to the way XCSF sets parameters for conditions, actions, and predictions,
# it is necessary to set the desired parameters in a wrapper function...


def get_prediction(n_init: int) -> dict:
    """Returns parameters for initialising neural network predictions."""
    return {
        "type": "neural",
        "args": {
            "layer_0": {
                "type": "connected",
                "activation": "selu",
                "n_init": n_init,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
            "layer_1": {
                "type": "connected",
                "activation": "selu",
                "n_init": Y_DIM,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
        },
    }


def get_model() -> xcsf.XCS:
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=X_DIM,
        y_dim=Y_DIM,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=200000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.1,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction=get_prediction(40),
    )

In [6]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

### Sklearn pipeline

Note: sklearn pipelines do not allow the use of callback functions.

In [7]:
# split (original data) into training and test sets
X_tr, X_te, y_tr, y_te = train_test_split(
    X_orig, y_orig, test_size=0.1, random_state=RANDOM_STATE
)

model = get_model()
model.set_params(max_trials=50000)  # speed this demo up

pipe = make_pipeline(
    StandardScaler(),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

pipe.fit(X_tr, y_tr)

2023-07-19T15:46:32.013 trials=0 train=1.86722 pset=500.0 mset=238.9 mfrac=0.65
2023-07-19T15:46:32.911 trials=5000 train=0.22114 pset=500.0 mset=223.5 mfrac=0.82
2023-07-19T15:46:34.440 trials=10000 train=0.18990 pset=500.0 mset=366.9 mfrac=1.00
2023-07-19T15:46:36.700 trials=15000 train=0.24111 pset=500.0 mset=405.6 mfrac=0.98
2023-07-19T15:46:38.594 trials=20000 train=0.14901 pset=500.0 mset=378.3 mfrac=0.96
2023-07-19T15:46:40.544 trials=25000 train=0.15173 pset=500.0 mset=414.2 mfrac=0.94
2023-07-19T15:46:42.130 trials=30000 train=0.13972 pset=500.0 mset=406.4 mfrac=0.37
2023-07-19T15:46:43.714 trials=35000 train=0.12362 pset=500.0 mset=394.7 mfrac=0.68
2023-07-19T15:46:45.590 trials=40000 train=0.11709 pset=500.0 mset=390.1 mfrac=0.87
2023-07-19T15:46:47.664 trials=45000 train=0.11693 pset=500.0 mset=384.0 mfrac=0.99


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7f9adca8a6f0>,
                                            transformer=StandardScaler()))])

In [8]:
train_score = pipe.score(X_tr, y_tr)
test_score = pipe.score(X_te, y_te)

print(f"train = {train_score}")
print(f"test = {test_score}")

train = 0.8870088321372127
test = 0.8812029081717271


### Sklearn grid search

In [9]:
# General parameters can be searched in the usual way

parameters = dict(beta=[0.1, 0.5])

# EA parameters require specifying a dict,
# but individual values can still be set because the other values are not changed.

parameters = dict(ea=[{"lambda": 2}, {"lambda": 10}])

# However, for actions, conditions, and predictions, the WHOLE dict must be specified
# for each value to try in the search. See Optuna below.

model = get_model()
model.set_params(max_trials=25000)  # speed this demo up

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring="neg_mean_squared_error",
    cv=3,
)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

2023-07-19T15:46:52.338 trials=0 train=1.93661 pset=500.0 mset=237.9 mfrac=0.62
2023-07-19T15:46:53.727 trials=5000 train=0.26937 pset=500.0 mset=230.3 mfrac=0.82
2023-07-19T15:46:55.220 trials=10000 train=0.20115 pset=500.0 mset=369.0 mfrac=0.62
2023-07-19T15:46:57.204 trials=15000 train=0.16725 pset=500.0 mset=403.1 mfrac=0.87
2023-07-19T15:46:59.688 trials=20000 train=0.17910 pset=500.0 mset=373.2 mfrac=0.81
2023-07-19T15:47:10.114 trials=0 train=2.61311 pset=500.0 mset=236.5 mfrac=1.00
2023-07-19T15:47:11.209 trials=5000 train=0.22297 pset=500.0 mset=213.3 mfrac=0.75
2023-07-19T15:47:12.621 trials=10000 train=0.23937 pset=500.0 mset=396.2 mfrac=0.97
2023-07-19T15:47:14.593 trials=15000 train=0.16062 pset=500.0 mset=413.3 mfrac=0.65
2023-07-19T15:47:16.916 trials=20000 train=0.14551 pset=500.0 mset=407.7 mfrac=0.66
2023-07-19T15:47:23.004 trials=0 train=2.04997 pset=500.0 mset=238.0 mfrac=0.98
2023-07-19T15:47:24.218 trials=5000 train=0.22114 pset=500.0 mset=217.2 mfrac=0.71
2023-07

### Optuna

In [10]:
def objective(trial):
    """Measure a new hyperparameter combination."""
    # get new parameters to try
    e0: float = trial.suggest_float("e0", 0.08, 0.12, step=0.01)
    beta: float = trial.suggest_float("beta", 0.01, 0.19, step=0.02)
    n_init: int = trial.suggest_int("n_init", 1, 101, step=10)
    # get a new XCSF and set params - predictions must include the whole dict
    prediction: dict = get_prediction(n_init=n_init)
    model = get_model()
    model.set_params(e0=e0, beta=beta, prediction=prediction)
    # get a new early stopping callback
    callback = get_callback()
    # fit XCSF
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )
    # optimise validation MSE
    return model.score(X_val, y_val)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))

[I 2023-07-19 15:48:03,850] A new study created in memory with name: no-name-65ccbe34-6e31-413a-8b44-5e9b590ad4f3


In [11]:
study.optimize(objective, n_trials=10)

2023-07-19T15:48:04.586 trials=0 train=0.66547 val=0.71319 pset=500.0 mset=278.6 mfrac=0.06
2023-07-19T15:48:04.587 checkpoint: 0.71319 error at 0 trials
2023-07-19T15:48:05.264 trials=5000 train=0.59617 val=0.60174 pset=500.0 mset=307.7 mfrac=0.86
2023-07-19T15:48:05.267 checkpoint: 0.60174 error at 5000 trials
2023-07-19T15:48:06.059 trials=10000 train=0.60200 val=0.60767 pset=500.0 mset=391.6 mfrac=0.31
2023-07-19T15:48:06.897 trials=15000 train=0.57202 val=0.58019 pset=500.0 mset=357.1 mfrac=0.01
2023-07-19T15:48:06.900 checkpoint: 0.58019 error at 15000 trials
2023-07-19T15:48:07.743 trials=20000 train=0.56980 val=0.60708 pset=500.0 mset=342.9 mfrac=0.00
2023-07-19T15:48:08.588 trials=25000 train=0.57959 val=0.60004 pset=500.0 mset=330.3 mfrac=0.22
2023-07-19T15:48:09.453 trials=30000 train=0.59414 val=0.57761 pset=500.0 mset=315.5 mfrac=0.00
2023-07-19T15:48:09.456 checkpoint: 0.57761 error at 30000 trials
2023-07-19T15:48:10.280 trials=35000 train=0.57111 val=0.60486 pset=500.0 

[I 2023-07-19 15:48:19,416] Trial 0 finished with value: 0.5776050574797028 and parameters: {'e0': 0.1, 'beta': 0.15000000000000002, 'n_init': 1}. Best is trial 0 with value: 0.5776050574797028.


2023-07-19T15:48:19.337 trials=55000 train=0.56789 val=0.59860 pset=500.0 mset=224.2 mfrac=0.13
2023-07-19T15:48:19.339 restoring system from trial 30000 with error=0.57761
2023-07-19T15:48:20.261 trials=0 train=2.94642 val=8.36705 pset=500.0 mset=214.5 mfrac=1.00
2023-07-19T15:48:20.263 checkpoint: 8.36705 error at 0 trials
2023-07-19T15:48:21.061 trials=5000 train=0.41665 val=0.29036 pset=500.0 mset=217.7 mfrac=1.00
2023-07-19T15:48:21.064 checkpoint: 0.29036 error at 5000 trials
2023-07-19T15:48:22.487 trials=10000 train=0.30444 val=0.28284 pset=500.0 mset=387.7 mfrac=1.00
2023-07-19T15:48:22.491 checkpoint: 0.28284 error at 10000 trials
2023-07-19T15:48:23.900 trials=15000 train=0.29390 val=2.57404 pset=500.0 mset=428.5 mfrac=1.00
2023-07-19T15:48:25.384 trials=20000 train=0.24860 val=0.21924 pset=500.0 mset=435.0 mfrac=1.00
2023-07-19T15:48:25.387 checkpoint: 0.21924 error at 20000 trials
2023-07-19T15:48:26.856 trials=25000 train=0.23821 val=0.28609 pset=500.0 mset=447.0 mfrac=1.

[I 2023-07-19 15:48:52,076] Trial 1 finished with value: 0.16324620544104507 and parameters: {'e0': 0.09, 'beta': 0.03, 'n_init': 11}. Best is trial 1 with value: 0.16324620544104507.


2023-07-19T15:48:51.968 trials=105000 train=0.16832 val=0.16348 pset=500.0 mset=171.1 mfrac=0.66
2023-07-19T15:48:51.971 restoring system from trial 80000 with error=0.16325
2023-07-19T15:48:53.906 trials=0 train=6.62380 val=0.25126 pset=500.0 mset=256.2 mfrac=1.00
2023-07-19T15:48:53.920 checkpoint: 0.25126 error at 0 trials
2023-07-19T15:48:55.200 trials=5000 train=0.33433 val=0.23027 pset=500.0 mset=243.1 mfrac=0.50
2023-07-19T15:48:55.204 checkpoint: 0.23027 error at 5000 trials
2023-07-19T15:48:56.959 trials=10000 train=0.19652 val=0.19619 pset=500.0 mset=382.4 mfrac=1.00
2023-07-19T15:48:56.964 checkpoint: 0.19619 error at 10000 trials
2023-07-19T15:48:58.915 trials=15000 train=0.63862 val=0.18836 pset=500.0 mset=366.2 mfrac=0.13
2023-07-19T15:48:58.919 checkpoint: 0.18836 error at 15000 trials
2023-07-19T15:49:00.822 trials=20000 train=0.18317 val=0.19169 pset=500.0 mset=348.7 mfrac=0.25
2023-07-19T15:49:02.715 trials=25000 train=0.19676 val=0.17326 pset=500.0 mset=353.3 mfrac=0

...

[I 2023-07-19 15:58:16,253] Trial 7 finished with value: 0.10419906184567795 and parameters: {'e0': 0.12, 'beta': 0.06999999999999999, 'n_init': 71}. Best is trial 7 with value: 0.10419906184567795.


2023-07-19T15:58:17.967 trials=0 train=0.68861 val=0.69311 pset=500.0 mset=243.8 mfrac=0.41
2023-07-19T15:58:17.969 checkpoint: 0.69311 error at 0 trials
2023-07-19T15:58:18.829 trials=5000 train=0.62427 val=0.61960 pset=500.0 mset=259.0 mfrac=0.49
2023-07-19T15:58:18.833 checkpoint: 0.61960 error at 5000 trials
2023-07-19T15:58:20.698 trials=10000 train=0.58836 val=0.68150 pset=500.0 mset=399.9 mfrac=0.74
2023-07-19T15:58:23.151 trials=15000 train=0.57764 val=0.63622 pset=500.0 mset=378.3 mfrac=0.21
2023-07-19T15:58:24.525 trials=20000 train=0.57469 val=0.60491 pset=500.0 mset=340.8 mfrac=0.00
2023-07-19T15:58:24.529 checkpoint: 0.60491 error at 20000 trials
2023-07-19T15:58:25.893 trials=25000 train=0.55018 val=0.61081 pset=500.0 mset=319.8 mfrac=0.00
2023-07-19T15:58:27.350 trials=30000 train=0.58060 val=0.60968 pset=500.0 mset=310.5 mfrac=0.00
2023-07-19T15:58:31.235 trials=35000 train=0.57227 val=0.60938 pset=500.0 mset=298.8 mfrac=0.91
2023-07-19T15:58:32.226 trials=40000 train=0

[I 2023-07-19 15:58:45,733] Trial 8 finished with value: 0.5834895718998513 and parameters: {'e0': 0.12, 'beta': 0.17, 'n_init': 1}. Best is trial 7 with value: 0.10419906184567795.


2023-07-19T15:58:45.673 trials=95000 train=0.56258 val=0.64163 pset=500.0 mset=139.3 mfrac=0.00
2023-07-19T15:58:45.677 restoring system from trial 70000 with error=0.58349
2023-07-19T15:58:48.748 trials=0 train=10.75420 val=0.42921 pset=500.0 mset=275.0 mfrac=1.00
2023-07-19T15:58:48.760 checkpoint: 0.42921 error at 0 trials
2023-07-19T15:58:52.445 trials=5000 train=0.50441 val=0.26475 pset=500.0 mset=305.7 mfrac=1.00
2023-07-19T15:58:52.453 checkpoint: 0.26475 error at 5000 trials
2023-07-19T15:58:57.069 trials=10000 train=0.68747 val=1.02319 pset=500.0 mset=484.3 mfrac=1.00
2023-07-19T15:59:02.168 trials=15000 train=0.35618 val=0.23635 pset=500.0 mset=486.5 mfrac=1.00
2023-07-19T15:59:02.175 checkpoint: 0.23635 error at 15000 trials
2023-07-19T15:59:07.385 trials=20000 train=0.33337 val=0.20877 pset=500.0 mset=487.7 mfrac=1.00
2023-07-19T15:59:07.392 checkpoint: 0.20877 error at 20000 trials
2023-07-19T15:59:12.609 trials=25000 train=0.27043 val=0.17449 pset=500.0 mset=490.0 mfrac=1

[I 2023-07-19 16:00:45,463] Trial 9 finished with value: 0.1235938095879669 and parameters: {'e0': 0.08, 'beta': 0.03, 'n_init': 91}. Best is trial 7 with value: 0.10419906184567795.


In [12]:
best_trial = study.best_trial
print(f"Best trial - Evaluation metric: {best_trial.value:.4f}")
print(f"Best trial - Hyperparameters: {best_trial.params}")

Best trial - Evaluation metric: 0.1042
Best trial - Hyperparameters: {'e0': 0.12, 'beta': 0.06999999999999999, 'n_init': 71}


In [13]:
plot_param_importances(study)